In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgbm
from lightgbm import *
pd.set_option("display.max_columns", None)

In [ ]:
df_train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
df_train.head()

In [ ]:
df_train.dtypes

In [ ]:
df_train.shape

In [ ]:
pd.isnull(df_train).sum()

In [ ]:
plt.figure(figsize=(20,20))
for i in range(15) :
    plt.subplot(5, 5, i+1)
    plt.hist(df_train[f"f_{i}"], bins=100)
    plt.title(f"f_{i}")
plt.show

In [ ]:
df_train.describe()

In [ ]:
feature_cols = df_train.columns.unique()
feature_cols = feature_cols.drop(["row_id", "time_id", "target"])
feature_cols

In [ ]:
outlier_list = []
outlier_col = []

for col in feature_cols:
    _mean, _std = df_train[col].mean(), df_train[col].std()
    
    tmp_df_1 = df_train[(df_train[col] > _mean + _std * 70) |
                     (df_train[col] < _mean - _std * 70)]
    tmp_df_2 = df_train[(df_train[col] > _mean + _std * 35) |
                     (df_train[col] < _mean - _std * 35)]
    if len(tmp_df_1) > 0 : 
        outliers = tmp_df_1.index.to_list()
        outlier_list.extend(outliers)
        outlier_col.append(col)
        print(col, len(tmp_df_1))
        
    elif len(tmp_df_2) > 0 and len(tmp_df_2) < 6 :
        outliers = tmp_df_2.index.to_list()
        outlier_list.extend(outliers)
        outlier_col.append(col)
        print(col, len(tmp_df_2))
    _mean, _std = None, None

outlier_list = list(set(outlier_list))
print(len(outlier_col), len(outlier_list))

In [ ]:
plt.figure(figsize=(20,20))
for i, (col) in enumerate(["f_165", "f_214", "f_295"]) :
    plt.subplot(5, 5, i+1)
    plt.scatter(df_train[col], df_train["target"])
    plt.title(col)
plt.show

In [ ]:
df_train.drop(df_train.index[outlier_list], inplace = True)
df_train

In [ ]:
plt.figure(figsize=(20,20))
for i, (col) in enumerate(["f_165", "f_214", "f_295"]) :
    plt.subplot(5, 5, i+1)
    plt.scatter(df_train[col], df_train["target"])
    plt.title(col)
plt.show

In [ ]:
plt.figure(figsize=(20,20))
for i in range(15) :
    plt.subplot(5, 5, i+1)
    plt.hist(df_train[f"f_{i}"], bins=100)
    plt.title(f"f_{i}")
plt.show

In [ ]:
def reduce_mem_usage(df):
  
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
     
    return df

df_train = reduce_mem_usage(df_train)

In [ ]:
X = df_train[feature_cols]
y = df_train['target'].copy()

In [ ]:
model_entire = lgbm.LGBMRegressor(
        objective="regression",
        metric="rmse",
        n_estimators=300 )

In [ ]:
%%time

model_entire.fit(X, y)